***
<font size="10"><b>VacationPy<b></font>
***

# SETUP

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import gkey

# weather data file path
file = os.path.join("..", "output_data/cities.csv")

# LOAD DATA

In [2]:
weather_df = pd.read_csv(file)
weather_df = pd.DataFrame(weather_df)
weather_df.head()

,City,CityID,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Portland,5746545,US,45.52,-122.68,48.20,87,90,8.05
1,Kodiak,5866583,US,57.79,-152.41,21.20,79,90,16.11
2,Mwinilunga,902620,ZM,-11.74,24.43,65.59,94,94,3.65
3,Port Macquarie,2152659,AU,-31.43,152.92,75.00,94,40,5.82
4,Port-Gentil,2396518,GA,-0.72,8.78,84.20,89,20,4.70


# HUMIDITY HEATMAP

In [3]:
# Configure Map
gmaps.configure(api_key = gkey)
fig_size = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout = fig_size)

In [4]:
# Set data
city_coords = weather_df[["Latitude","Longitude"]]
city_hum = weather_df["Humidity"].astype(float)

In [5]:
# Add Heatlayer to Map
heat_layer = gmaps.heatmap_layer(city_coords, weights = city_hum)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

# NARROWING WEATHER CONDITIONS

In [6]:
cities_pref = weather_df.loc[(weather_df["Temperature"]<85)&
                           (weather_df["Temperature"]>65)&
                           (weather_df["Cloudiness"]<20)&
                           (weather_df["Wind Speed"]<10)
                          ]
cities_pref = cities_pref.dropna()
cities_pref.head()

,City,CityID,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
42,Valparaiso,3868626,CL,-33.04,-71.63,68.00,68,0,9.17
44,Kapaa,5848280,US,22.08,-159.32,66.20,88,1,6.93
48,Maragogi,3395458,BR,-9.01,-35.22,82.35,66,0,9.06
57,Kawerau,2188942,NZ,-38.10,176.70,68.00,92,1,2.17
78,Sittwe,1295765,MM,20.15,92.90,70.81,59,0,5.59


In [7]:
cities_pref.shape

(29, 9)

# HOTEL MAP

In [8]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

hotels = []
params = {"key": gkey,
          "radius": "5000",
          "type": "lodging"
         }

for index,row in cities_pref.iterrows():    
    params['location']=str(row['Latitude'])+","+str(row['Longitude'])    
    response = requests.get(url,params = params).json()

    try:
        hotel_name = (response["results"][0]["name"])
        hotel_lat = (response["results"][0]["geometry"]["location"]["lat"])
        hotel_lng = (response["results"][0]["geometry"]["location"]["lng"])
        hotels.append({"Hotel Name":hotel_name,
                       "City":row["City"],
                       "Country":row["Country"],
                       "Hotel_Lat":hotel_lat,
                       "Hotel_Lng":hotel_lng
                      }
                     )
    except(IndexError):
        print(row["City"])

Oum Hadjer
Mount Darwin
La Reforma


In [9]:
hotels_df = pd.DataFrame(hotels)
hotels_df.head()

,Hotel Name,City,Country,Hotel_Lat,Hotel_Lng
0,Hotel Ultramar,Valparaiso,CL,-33.048771,-71.632431
1,Sheraton Kauai Coconut Beach Resort,Kapaa,US,22.056750,-159.322476
2,Salinas do Maragogi All Inclusive Resort,Maragogi,BR,-9.027939,-35.227731
3,Tui Cottage,Kawerau,NZ,-38.097198,176.687943
4,Hotel Memory,Sittwe,MM,20.140510,92.899216


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Hotel_Lat", "Hotel_Lng"]]

In [11]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

# APPENDIX